In [2]:
%run model_sucrose.ipynb

Computing gene knockout strategies using the OptGene algorithm. The OptGene algorithm is based on an evolutionary approach which suggests several potential knockouts of different genes meaning that it is possible to screen for a fitting mutant among these options (Rocha et al., 2008). 

## Preparations

Import the packages which are necessary for OptGene:

In [21]:
import cameo

In [22]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [23]:
from cameo import models

Basing algorithms upon the surfactin reaction which is based in the fatty acid that is most utilized. Looking at the reaction stoichiometry again:

In [43]:
print(model.reactions.SP_1)

SP_1: 3hddcoa_c + asp__L_c + atp_c + glu__L_c + 4 leu__L_c + nadph_c + val__L_c --> adp_c + nadp_c + surfactin_c


Finding the other reaction metabolite names that are needed:

In [44]:
model.reactions.query("biomass","name") #Biomass reaction name

[<Reaction BIOMASS_BS_10 at 0x1cf9b238f10>]

In [45]:
model.metabolites.query("sucr") #Glucose metabolite name

[<Metabolite sucr_c at 0x1cf9142d430>, <Metabolite sucr_e at 0x1cf9142d580>]

Changing objective to surfactin:

In [46]:
model.objective = model.reactions.SP_1

Import the packages which are necessary for OptGene:

## OptGene Algorithm

Running the OptGene algorithm to search for gene or reaction knockouts:

In [50]:
optgene = OptGene(model)

In [51]:
result_optgene = optgene.run(target="SP_1", #Reaction
                     biomass="BIOMASS_BS_10", #Reaction
                     substrate="sucr_e", #Metabolite
                     max_evaluations=20000, max_knockouts=3, plot=False)

Starting optimization at Wed, 24 Nov 2021 23:07:41


HBox()

Finished after 00:12:59


In [52]:
result_optgene

HBox()

,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness
0,"(CYTB_B3,)","((BSU38140,), (BSU38160,), (BSU38150,), (BSU38...",1,0.0,0.003571,0.0,0.624185,0.0,0.0
1,"(GART,)","((BSU02230,),)",1,0.0,0.003571,0.0,0.624185,0.0,0.0
2,"(CYTB_B3, CDGPT_BS)","((BSU38160, BSU16920),)",2,0.0,0.343651,0.0,0.023593,0.0,0.0
3,"(CYTB_B3, UGT2_BS, UGT_BS, UGT1_BS)","((BSU38160, BSU21920),)",2,0.0,0.343740,0.0,0.023593,0.0,0.0
4,"(UGT1_BS, UGT2_BS, UGT_BS, CYTB_B3)","((BSU21920, BSU38170),)",2,0.0,0.343740,0.0,0.023593,0.0,0.0
5,"(UGT1_BS, UGT_BS, CYTB_B3, CDGPT_BS, UGT2_BS)","((BSU38160, BSU16920, BSU21920),)",3,0.0,0.343740,0.0,0.023593,0.0,0.0


Several suggestions of reactions and genes were suggested by the OptGene algorithm.

Double checking these result by manually knock outs and computng the surfactin yield:

In [14]:
with model as mutant0:
    mutant0.reactions.CYTB_B3.bounds = 0, 0
    print("CYTB_B3's bounds inside the with statement:")
    print(mutant0.reactions.CYTB_B3.lower_bound, mutant0.reactions.CYTB_B3.bounds)
    mutant0.objective=mutant0.reactions.DM_surfactin_c
    surfactin_production = mutant0.optimize().objective_value #flux
    print('Maximum productivity =', surfactin_production, 'mmol/gDW*h')
    maximum_yield = surfactin_production / (-1*(mutant0.reactions.EX_sucr_e.flux)) #yield calculations
    print('Maximum theoretical yield =', maximum_yield, 'mmol-surfactin/mmol-sucrose')


CYTB_B3's bounds inside the with statement:
0 (0, 0)
Maximum productivity = 0.3436433767512171 mmol/gDW*h
Maximum theoretical yield = 0.12160567339856444 mmol-surfactin/mmol-sucrose


In [19]:
with model as mutant1:
    mutant1.genes.BSU02230.knock_out()
    print("Maximum productivity =", mutant1.slim_optimize(), 'mmol/gDW*h')
    mutant1.objective = mutant1.reactions.DM_surfactin_c
    surfactin_production = mutant1.optimize().objective_value
    print("Maximum theoretical yield =", surfactin_production / (-1*mutant1.reactions.EX_sucr_e.flux), 'mmol-surfactin/mmol-sucrose')

Maximum productivity = 0.3436433767512166 mmol/gDW*h
Maximum theoretical yield = 0.1217679725107815 mmol-surfactin/mmol-sucrose


In [20]:
with model as mutant2:
    mutant2.genes.BSU21920.knock_out()
    mutant2.genes.BSU16920.knock_out()
    print("Maximum productivity =", mutant2.slim_optimize(), 'mmol/gDW*h')
    mutant2.objective = mutant2.reactions.DM_surfactin_c
    surfactin_production = mutant2.optimize().objective_value
    print("Maximum theoretical yield =", surfactin_production / (-1*mutant2.reactions.EX_sucr_e.flux), 'mmol-surfactin/mmol-sucrose')

Maximum productivity = 0.34374044623728484 mmol/gDW*h
Maximum theoretical yield = 0.12195609188978644 mmol-surfactin/mmol-sucrose


In [25]:
with model as mutant3:
    mutant3.genes.BSU38160.knock_out()
    mutant3.genes.BSU21920.knock_out()
    print("Maximum productivity =", mutant3.slim_optimize(), 'mmol/gDW*h')
    mutant3.objective = mutant3.reactions.DM_surfactin_c
    surfactin_production = mutant3.optimize().objective_value
    print("Maximum theoretical yield =", surfactin_production / (-1*mutant3.reactions.EX_sucr_e.flux), 'mmol-surfactin/mmol-sucrose')

Maximum productivity = 0.34374044623728583 mmol/gDW*h
Maximum theoretical yield = 0.12037141448071839 mmol-surfactin/mmol-sucrose


In [26]:
with model as mutant4:
    mutant4.genes.BSU21920.knock_out()
    mutant4.genes.BSU38170.knock_out()
    print("Maximum productivity =", mutant4.slim_optimize(), 'mmol/gDW*h')
    mutant4.objective = mutant4.reactions.DM_surfactin_c
    surfactin_production = mutant4.optimize().objective_value
    print("Maximum theoretical yield =", surfactin_production / (-1*mutant4.reactions.EX_sucr_e.flux), 'mmol-surfactin/mmol-sucrose')

Maximum productivity = 0.34374044623728583 mmol/gDW*h
Maximum theoretical yield = 0.12037141448071839 mmol-surfactin/mmol-sucrose


In [27]:
with model as mutant5:
    mutant5.genes.BSU38160.knock_out()
    mutant5.genes.BSU16920.knock_out()
    mutant5.genes.BSU21920.knock_out()
    print("Maximum productivity =", mutant5.slim_optimize(), 'mmol/gDW*h')
    mutant5.objective = mutant5.reactions.DM_surfactin_c
    surfactin_production = mutant5.optimize().objective_value
    print("Maximum theoretical yield =", surfactin_production / (-1*mutant5.reactions.EX_sucr_e.flux), 'mmol-surfactin/mmol-sucrose')

Maximum productivity = 0.3437404462372878 mmol/gDW*h
Maximum theoretical yield = 0.12048121707873301 mmol-surfactin/mmol-sucrose


We see that computing each of these knockout suggestions leads to an increase in surfactin yield. The knockouts by mutant2 seems to have given the biggest increase in yield.